In [15]:
import sys
path='/home/tomas/Ulmer-Berechnung/alps2qutipplus-april/alps2qutipplus-main/'

sys.path.insert(1, path) 


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qutip
import scipy.linalg as linalg
import time
from multiprocessing import Pool
from itertools import product
from typing import Optional

from alpsqutip import (build_system, list_models_in_alps_xml,
                       list_geometries_in_alps_xml, graph_from_alps_xml,
                       model_from_alps_xml,
                       restricted_maxent_toolkit as me)

from alpsqutip.operators.states.utils import safe_exp_and_normalize ## function used to safely and robustly map K-states to states

from alpsqutip.scalarprod.build import fetch_covar_scalar_product
from alpsqutip.scalarprod.orthogonalize import orthogonalize_basis
from alpsqutip.operators.states.gibbs import GibbsDensityOperator, GibbsProductDensityOperator

In [17]:
from alpsqutip.operators.states.meanfield.projections import (one_body_from_qutip_operator, 
                                                  project_operator_to_m_body, 
                                                             project_qutip_operator_to_m_body,
                                                             project_to_n_body_operator,
                                                             project_qutip_operator_as_n_body_operator,
                                                             project_product_operator_as_n_body_operator)

In [18]:

from alpsqutip.operators.arithmetic import (
    ScalarOperator,
    LocalOperator,
    OneBodyOperator,
    Operator,
    ProductOperator,
    ScalarOperator,
    SumOperator,
    QutipOperator
)

from alpsqutip.operators.simplify import simplify_sum_operator

In [19]:
params={}

params['size']=6
params['Jx']=1.; params['Jy'] = .75*params['Jx']; params['Jz']=1.05*params['Jx']

from scipy.optimize import root, fsolve
Ffactor=np.real(max(np.roots(np.poly1d([1, 0, -(params['Jx']*params['Jy']+params['Jx']*params['Jy']+params['Jy']*params['Jz']), 
                           -2*params['Jx']*params['Jy']*params['Jz']]))))
chi_y=fsolve(lambda x,y: x*np.arcsinh(x)-np.sqrt(x**2+1)-y, 1e-1, args=(0))[0]
vLR=4*Ffactor*chi_y



In [20]:
system=build_system(geometry_name= "open chain lattice",model_name="spin", 
                    L=params['size'], J=1)

sites=[s for s in system.sites]
sx_ops=[system.site_operator("Sx", '1[' + str(a) + ']') for a in range(len(system.sites))]
sy_ops=[system.site_operator("Sy", '1[' + str(a) + ']') for a in range(len(system.sites))]
sz_ops=[system.site_operator("Sz", '1[' + str(a) + ']') for a in range(len(system.sites))]

idop = [system.site_operator('identity@1[' + str(a) + ']') for a in range(len(system.sites))]
from functools import reduce
idop = reduce(Operator.__mul__, idop)

#Iz = sum(sz_ops)
H_nn = 0  # Proches voisins
H_lr = 0  # Longue portée

from itertools import combinations

for i, j in combinations(range(params['size']), 2):
    r = abs(i - j)
    Jx_ij = params['Jx'] / r**3
    Jy_ij = params['Jy'] / r**3
    Jz_ij = params['Jz'] / r**3

    term = (
        Jx_ij * sx_ops[i] * sx_ops[j]
        + Jy_ij * sy_ops[i] * sy_ops[j]
        + Jz_ij * sz_ops[i] * sz_ops[j]
    )

    if r == 1:
        H_nn += term  # Interaction de voisins immédiats
    else:
        H_lr += term  # Interaction à longue portée
        
H = H_nn + H_lr
H = H.simplify()

loading model spin  over graph open chain lattice


In [21]:
z = 0.738259088061272

In [22]:
obs_SzA = sum(sz for sz in sz_ops)
HBB0=[idop]+[sz for sz in sz_ops]

phi0 = np.array([.0] + [-1.5+z/10 for _ in HBB0[1:]])

K0 = me.k_state_from_phi_basis(phi0, HBB0).simplify()
sigma0 = GibbsProductDensityOperator(K0)
phi0[0] = np.log(sigma0.tr())
K0 = me.k_state_from_phi_basis(phi0, HBB0).simplify()
sigma0 = GibbsProductDensityOperator(K0)

tgt_obs = obs_SzA
[(sigma0 * op).tr().real for op in sz_ops] 

[0.3063044948165552,
 0.3063044948165552,
 0.3063044948165552,
 0.3063044948165552,
 0.3063044948165552,
 0.3063044948165552]

In [23]:
from concurrent.futures import ProcessPoolExecutor
from itertools import product
from alpsqutip.operators import ScalarOperator, SumOperator

def _commutator_term_worker(hi_kj):
    hi, kj = hi_kj
    comm = (hi * kj - kj * hi).simplify()
    return comm
    
def _projection_worker(args):
    term, m_max, sigma_0 = args
    return project_to_n_body_operator(
        operator=term,
        nmax=m_max,
        sigma=sigma_0,
    ).simplify().tidyup(1e-10)

def parallelized_real_time_projection_of_hierarchical_basis(
    generator, 
    seed_op,
    sigma_ref,
    m_max, 
    deep,
    num_workers=None,
    tidy_thresh=1e-5,
):
    if seed_op is None or deep == 0:
        return []

    basis = [seed_op]
    gen_terms = (-1j * generator).as_sum_of_products().terms
    system = generator.system

    for i in range(1, deep):
        current_op = basis[-1]
        basis_last_terms = (
            current_op.terms if isinstance(current_op, SumOperator)
            else current_op.as_sum_of_products().terms
        )

        term_pairs = list(product(gen_terms, basis_last_terms))

        with ProcessPoolExecutor(max_workers=num_workers) as executor:
            commutator_terms = list(executor.map(_commutator_term_worker, term_pairs, chunksize=128))

        commutator_terms = [term for term in commutator_terms if term is not None]
        projection_args = [(term, m_max, sigma_ref) for term in commutator_terms]

        with ProcessPoolExecutor(max_workers=num_workers) as executor:
            projected_terms = list(executor.map(_projection_worker,
                                                projection_args, chunksize=128))

        local_op = sum(projected_terms, ScalarOperator(0, system=system))
        local_op = (local_op).simplify()

        # --- Debug check: small norm triggers early exit ---
        norm_val = local_op.to_qutip().norm()
        #print(f"[DEBUG] Depth {i}: projected norm = {norm_val:.3e}")
        if norm_val < 1e-10:
        #    print("[DEBUG] Basis update terminated early: projected operator norm too small.")
            break

        basis.append(local_op)

    return basis

def _hij_worker_explicit(args):
    """
    Compute (i, val) where val = Re⟨b_i | [H, b_last]_proj⟩.

    Steps:
        - Compute commutator: [H, b_last] = -i H b_last + i b_last H
        - Project the commutator onto the m_max-body subspace using sigma_0 as reference
        - Take the scalar product between b_i and the projected commutator

    Parameters:
        args (tuple): Contains the following elements:
            i (int): Index of the current basis element.
            basis_i (Operator): The i-th basis element b_i.
            b_last (Operator): The operator b_j being commuted with H.
            H (Operator): The system Hamiltonian.
            sp (callable): Scalar product function sp(op1, op2) -> complex.
            sigma_0 (Operator or State): Reference operator for projection.
            m_max (int): Max number of bodies to retain in the projection.

    Returns:
        tuple: (i, val) where val is Re⟨b_i | [H, b_last]_proj⟩.
    """
    i, basis_i, b_last, H, sp, sigma_0, m_max = args

    # Compute commutator [H, b_last]
    comm = (-1j * H * b_last + 1j * b_last * H).simplify()

    # Project onto the m-body subspace
    comm_proj = project_to_n_body_operator(
        operator=comm,
        nmax=m_max,
        sigma=sigma_0
    )

    # Compute scalar product with basis_i
    val = sp(basis_i, comm_proj)
    return i, val

In [24]:
def parallel_gram_schmidt_with_gram(basis, sp: callable, tol=1e-6, num_threads=4):
    """
    Parallel Gram-Schmidt orthogonalization with Gram matrix computation.

    Args:
        basis (List[Operator]): Initial (non-orthogonal) operator basis.
        sp (Callable): Scalar product function.
        tol (float): Threshold to drop near-zero vectors.
        num_threads (int): Threads for parallel scalar product computation.

    Returns:
        orth_basis (List[Operator]): Orthonormalized operator basis {q_k}.
        R (np.ndarray): Coefficients matrix s.t. q_k = sum_j R[k,j] * b_j.
        G (np.ndarray): Gram matrix G[i,j] = sp(b_i, b_j).
    """

    def ensure_consistent_system(op, system):
        op.system = system
        if hasattr(op, "terms"):
            for term in op.terms:
                term.system = system
        return op

    n = len(basis)
    system = getattr(basis[0], "system", None)

    # Compute full Gram matrix G in parallel
    G = np.zeros((n, n), dtype=np.complex128)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Upper triangular indices only
        pairs = [(i, j) for i in range(n) for j in range(i, n)]
        results = list(executor.map(lambda ij: (ij[0], ij[1], sp(basis[ij[0]], basis[ij[1]])), pairs))
    for i, j, val in results:
        G[i, j] = val
        if i != j:
            G[j, i] = np.conj(val)  # Hermitian

    orth_basis = []
    R_rows = []

    for k, b_k in enumerate(basis):
        q_k = deepcopy(b_k)
        ensure_consistent_system(q_k, system)

        Rk = np.zeros(n, dtype=np.complex128)
        Rk[k] = 1.0

        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            projections = list(executor.map(
                lambda q_j: sp(q_j, q_k),
                orth_basis
            ))

        for j, (proj, q_j) in enumerate(zip(projections, orth_basis)):
            q_k = q_k - proj * q_j
            Rk -= proj * R_rows[j]

        norm = np.real(sp(q_k, q_k))**0.5
        if norm < tol:
            continue

        q_k = q_k / norm
        Rk = Rk / norm

        ensure_consistent_system(q_k, system)
        orth_basis.append(q_k)
        R_rows.append(Rk)

    R = np.array(R_rows)

    return orth_basis, R, G

In [25]:
def real_time_projection_of_hierarchical_basis(generator, 
                                               seed_op,
                                               sigma_ref,
                                               nmax, 
                                               deep):
    basis = []
    if seed_op is not None and deep > 0:
        basis += [seed_op]
        
        for i in range(1, deep):
            local_op = -1j*me.commutator(generator.to_qutip_operator(), basis[-1].to_qutip_operator())
            basis.append(project_to_n_body_operator(operator = local_op.simplify().as_sum_of_products(),
                                                      nmax = nmax, 
                                                      sigma = sigma_ref).simplify())
    return basis

In [26]:
HBB_ell_act_prl = parallelized_real_time_projection_of_hierarchical_basis(
                    generator = H, 
                    seed_op = obs_SzA, 
                    sigma_ref = sigma0, 
                    m_max = 10,
                    deep = 4
                    )

In [27]:
HBB_ell_act_non_prl = real_time_projection_of_hierarchical_basis(
                        generator = H,
                        seed_op = obs_SzA,
                        sigma_ref = sigma0,
                        nmax = 10, 
                        deep = 4
                        )

In [28]:
sp_local = fetch_covar_scalar_product(sigma0)
sp_local(HBB_ell_act_prl[0], HBB_ell_act_prl[0])

4.314673306344753

In [29]:
(sigma0 * HBB_ell_act_prl[0] * HBB_ell_act_prl[0]).tr()

4.314673306344753

### 1. Test 1: Here I am testing that the parallelized and non-parallelized projected bases yield the same

In [30]:
[(op-qop).to_qutip().norm() for op,qop in zip(HBB_ell_act_prl, HBB_ell_act_non_prl)]

[0.0, 8.330547606693865e-15, 1.705825130357951e-14, 2.8508567221834965e-14]

### 1.bis Test 1.bis: Here I am testing that there is no problem with how sum_operators are being summed up, since these should yield the same results as before

In [31]:
[(op.to_qutip()-qop.to_qutip()).norm() for op,qop in zip(HBB_ell_act_prl, HBB_ell_act_non_prl)]

[0.0, 8.330547606693865e-15, 1.7046512140831324e-14, 2.8495873553652083e-14]

In [32]:

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from copy import deepcopy

In [33]:
sp_local = fetch_covar_scalar_product(sigma=sigma0)

In [34]:
start = time.time()
orth_basis_act_prl, R, Gram = parallel_gram_schmidt_with_gram(basis = HBB_ell_act_prl, 
                                              sp = sp_local, num_threads = 8)

print(time.time() - start)

10.98222017288208


In [35]:
orth_basis_act_non_prl = orthogonalize_basis(basis = HBB_ell_act_non_prl, 
                                             sp = sp_local)

### 2. Test 2: Here I am testing that the parallelized and non-parallelized orthogonalizations yield the same results

In [36]:
[(op.to_qutip()-qop.to_qutip()).norm() for op,qop in zip(orth_basis_act_prl, orth_basis_act_non_prl)]

[0.0, 2.0857651568384745e-14, 6.651029120000482e-14, 1.995575051133275e-12]

### 2.bis Test 2.bis: Same as in test 1.bis

In [37]:
[(op-qop).to_qutip().norm() for op,qop in zip(orth_basis_act_prl, orth_basis_act_non_prl)]

[0.0, 2.0857651568384745e-14, 6.645351059060844e-14, 1.99576115891999e-12]

### Test 3. From linear algebra, we know that $C = R B$ where C is the orthonormal basis and B is the original Hierarchical Basis, i.e.

$B=R^{-1}C$

In [38]:
[(linalg.inv(R)[i] @ orth_basis_act_prl - HBB_ell_act_prl[i]).to_qutip().norm()
 for i in range(len(orth_basis_act_prl))] 

[0.0, 0.0, 0.0, 0.0]

### Test 4. Here, I want to construct the Hij from previous inputs, using no parallelized function, I calculate everything by hand. First I want to prove that 

Hij_orth = R @ Hij_non-orth @ R.tranpose()

In [39]:
Hij_tensor_act_non_prl, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                                basis=HBB_ell_act_non_prl,
                                                                sp=sp_local)

In [40]:
Hij_tensor_act_orth_prl, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                                basis=orth_basis_act_non_prl,
                                                                sp=sp_local)

In [41]:
(qutip.Qobj(R @ Hij_tensor_act_non_prl @ R.transpose()) - qutip.Qobj(Hij_tensor_act_orth_prl)).norm()

1.9981910196139423e-13

In [55]:
eigenvalues, V = linalg.eig(Gram)

In [69]:
eigenvalues, V = linalg.eig(Gram)
orth_basis_try = [sum(V[j][i] * 1/np.sqrt(eigenvalues[i]) * HBB_ell_act_prl[j] for j in range(len(eigenvalues))).simplify()
                  for i in range(len(HBB_ell_act_prl))]

In [71]:
Hij_tensor_act_try, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                                basis=orth_basis_try,
                                                                sp=sp_local)

In [73]:
qutip.Qobj(Hij_tensor_act_try)

Quantum object: dims=[[4], [4]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.          0.         -0.18058227  0.15550366]
 [ 0.          0.         -0.27185495 -1.98974684]
 [ 0.08853629  0.2656585   0.          0.        ]
 [-0.07403983  1.99397614  0.          0.        ]]

In [75]:
qutip.Qobj(Hij_tensor_act_orth_prl)

Quantum object: dims=[[4], [4]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.00000000e+00 -2.34943394e-01  0.00000000e+00  8.96656092e-03]
 [ 1.12331302e-01  0.00000000e+00 -9.98134836e-01  0.00000000e+00]
 [ 0.00000000e+00  9.91007450e-01  0.00000000e+00 -1.74711602e+00]
 [-3.46944695e-17  0.00000000e+00  1.74669528e+00  0.00000000e+00]]

### Test 5. Now that that is clear, I want to prove that 

$Hij-non-orth[i][j] = Gram[i][j+1]$ and I only need to project the last column, $sp(bi, b[ell])$

In [42]:
n = len(HBB_ell_act_non_prl)
Hij_tensor_recursive = np.zeros((n, n), dtype=np.complex128)

# Fill all columns except the last one using j+1
for i in range(n):
    for j in range(n - 1):
        Hij_tensor_recursive[i, j] = Gram[i, j+1]
        #sp_local(HBB_ell_act_non_prl[i], HBB_ell_act_non_prl[j + 1])

# Prepare args for the last column (using _hij_worker_explicit)
args = [
    (i, HBB_ell_act_non_prl[i], HBB_ell_act_non_prl[-1], H, sp_local, sigma0, 3)
    for i in range(n)
]

# Call _hij_worker_explicit (returns (i, val)) and store val
for i in range(n):
    _, val = _hij_worker_explicit(args[i])
    Hij_tensor_recursive[i, n - 1] = val


In [43]:
qutip.Qobj(Hij_tensor_recursive - Hij_tensor_act_non_prl).norm()

1.9660043543699966e-14

In [60]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor

def _compute_sp(args):
    i, j, basis, sp = args
    val = sp(basis[i], basis[j])
    return i, j, val

def cholesky_orthogonalize_basis_parallel_process(basis, sp: callable, tol=1e-10, max_workers=None):
    """
    Parallel Cholesky orthogonalization using ProcessPoolExecutor.
    
    Args:
        basis (List[Operator]): Input operator basis {b_i}.
        sp (Callable): Scalar product function sp(op1, op2) -> complex.
        tol (float): Stabilization threshold.
        max_workers (int or None): Number of processes.

    Returns:
        orth_basis (List[Operator]): Orthonormalized basis {q_i}.
        R (np.ndarray): Upper-triangular matrix, b_j = sum_i R[i,j] * q_i.
        G (np.ndarray): Hermitian Gram matrix G[i,j] = sp(b_i, b_j).
    """
    n = len(basis)
    G = np.zeros((n, n), dtype=np.complex128)

    # Prepare arguments for processes
    args = [(i, j, basis, sp) for i in range(n) for j in range(i, n)]

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(_compute_sp, args)

    for i, j, val in results:
        G[i, j] = val
        if i != j:
            G[j, i] = np.conj(val)

    G = (G + G.conj().T) / 2  # Hermitize
    G += tol * np.eye(n)      # Stabilize

    try:
        R_dag = np.linalg.cholesky(G)  # Lower-triangular
    except np.linalg.LinAlgError:
        raise ValueError("Gram matrix is not positive-definite")

    R = R_dag.conj().T  # Upper-triangular: G = R† R

    R_inv = np.linalg.inv(R)
    orth_basis = [
        sum(R_inv[j, i] * basis[j] for j in range(n))
        for i in range(n)
    ]

    return orth_basis, R, G


In [31]:
orth_basis_cho, Rcho, Gcho = cholesky_orthogonalize_basis(basis=HBB_ell_act_non_prl, sp = sp_local)

In [49]:
diffs = [
    (sum(orth_basis_cho[j] * Rcho[j, i]  for j in range(len(orth_basis_cho))) - HBB_ell_act_non_prl[i]).to_qutip().norm()
    for i in range(len(orth_basis_cho))
]

diffs

[0.0, 0.0, 1.805328789133384e-15]

In [50]:
Hij_tensor_act_orth_prl

array([[ 0.        +0.j, -0.16481177+0.j,  0.        +0.j],
       [ 0.16481177+0.j,  0.        +0.j, -0.93419174+0.j],
       [ 0.        +0.j,  0.93419174+0.j,  0.        +0.j]])